In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import time 

In [12]:
browser = Browser('chrome')

In [13]:
states = [
    {'abbrev': 'MN', 'name': 'Minnesota'},
    {'abbrev': 'NY', 'name': 'New York'},
    {'abbrev': 'IL', 'name': 'Illinois'},
    {'abbrev': 'IN', 'name': 'Indiana'},
    {'abbrev': 'MI', 'name': 'Michgan'},
    {'abbrev': 'OH', 'name': 'Ohio'},
    {'abbrev': 'PA', 'name': 'Pennslyvania'},
    {'abbrev': 'WI', 'name': 'Wisconsin'}
]

In [46]:
base_url = 'https://ecos.fws.gov/ecp/report/species-listings-by-state'

for state in states:
    url = f"{base_url}?stateAbbrev={state['abbrev']}&stateName={state['name']}&statusCategory=Listed"
    
    browser.visit(url)
    browser.find_by_xpath("//select[@name='species-listings-by-state-report_length']/option[text()='All']").click()

    html_content = browser.html  


    # trying to scrap for the href and animal name 
    
    soup = BeautifulSoup(html_content,'html.parser')

    test = soup.find_all('tbody')
    for tag in test:
        url2 = tag.find_all('a', href = True)
        

        for tag2 in url2:
            url3 = tag2.find_all('a', href = True)
            print(url3)

    data = {'Name':[],'Href':[]}

    for item in soup.find_all('td', class_='sorting_2'):
            data['Name'].append(item.text)

    for item in soup.find_all('tbody'):    
        anchor = item.find('a',href=True)
        print(anchor['href'])




        

#     for item in soup.find_all()
    # soup = BeautifulSoup(html_content, 'html.parser')
    # image_elements = soup.find_all('img', class_='imageSize')

    # for image in image_elements:
    #     src = image['src'].strip()


[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
/ecp/species/6039
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
/ecp/species/6039
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
/ecp/species/6039
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
/ecp/species/10477
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
/ecp/species/6039
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
/ecp/species/6039
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
/ecp/species/6039
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
/ecp/species/6039


In [39]:
test2

<a href="/ecp/species/6039" ref="noopener noreferrer" target="_blank">Charadrius melodus</a>

In [42]:
test

[<tbody><tr class="dtrg-group dtrg-start dtrg-level-0"><td colspan="5">Birds</td></tr><tr class="odd"><td><a href="/ecp/species/6039" ref="noopener noreferrer" target="_blank">Charadrius melodus</a></td><td class="sorting_2">Piping Plover</td><td>[Great Lakes watershed DPS] - Great Lakes, watershed in States of IL, IN, MI, MN, NY, OH, PA, and WI and Canada (Ont.)</td><td>3</td><td>Endangered</td></tr><tr class="even"><td><a href="/ecp/species/1864" ref="noopener noreferrer" target="_blank">Calidris canutus rufa</a></td><td class="sorting_2">rufa red knot</td><td>Wherever found</td><td>5</td><td>Threatened</td></tr><tr class="odd"><td><a href="/ecp/species/758" ref="noopener noreferrer" target="_blank">Grus americana</a></td><td class="sorting_2">Whooping crane</td><td>U.S.A. (AL, AR, CO, FL, GA, ID, IL, IN, IA, KY, LA, MI, MN, MS, MO, NC, NM, OH, SC, TN, UT, VA, WI, WV, western half of WY)</td><td>2</td><td>Experimental Population, Non-Essential</td></tr><tr class="dtrg-group dtrg-star

In [25]:
data

{'Name': ['Piping Plover',
  'rufa red knot',
  'Whooping crane',
  'Higgins eye (pearlymussel)',
  'Sheepnose Mussel',
  'Snuffbox mussel',
  'Spectaclecase (mussel)',
  'Winged Mapleleaf',
  'Dwarf lake iris',
  'Eastern prairie fringed orchid',
  "Fassett's locoweed",
  "Mead's milkweed",
  'Northern wild monkshood',
  "Pitcher's thistle",
  'Prairie bush-clover',
  'bog buck moth',
  "Hine's emerald dragonfly",
  'Karner blue butterfly',
  'Poweshiek skipperling',
  'Rusty patched bumble bee',
  'Canada Lynx',
  'Gray wolf',
  'Northern Long-Eared Bat',
  'Eastern Massasauga (=rattlesnake)',
  'Iowa Pleistocene snail'],
 'Href': []}